# Get data about Corona Virus
source = https://github.com/CSSEGISandData/COVID-19
This data feeds this great dashboard: https://coronavirus.jhu.edu/map.html

In [12]:
import pandas as pd
import plotly.graph_objects as go
import plotly.express as px
import math
import numpy as np

import gspread
from oauth2client.service_account import ServiceAccountCredentials
import pygsheets
from datetime import date, timedelta

## Get data from John's Hopkins

In [13]:
start_date = date(2020, 1, 22)
end_date = date.today()

day = timedelta(days=1)

"{date.month:02}-{date.day:02}-{date.year}".format(date=start_date+day)

'01-23-2020'

In [14]:
dates_list = []

mydate = start_date
while mydate <= end_date:
    dates_list.append("{date.month:02}-{date.day:02}-{date.year}".format(date=mydate))
    mydate += day

In [15]:
df_list = []

for i in dates_list:
    df = pd.read_html('https://github.com/CSSEGISandData/COVID-19/blob/master/csse_covid_19_data/csse_covid_19_daily_reports/{}.csv'.format(i))[0]
    df['observation_date'] = i
    df_list.append(df)

HTTPError: HTTP Error 404: Not Found

In [ ]:
df = pd.concat(df_list, ignore_index=True, sort=False)

In [ ]:
df_group = df.groupby(by=['Country/Region','observation_date']
                      , as_index=False).agg({'Confirmed':'sum',
                                             'Deaths':'sum',
                                             'Recovered':'sum'})

In [ ]:
country_dict = {'Iran (Islamic Republic of)':'Iran',
                'Korea, South':'South Korea',
                'Republic of Korea':'South Korea',
                'Mainland China':'China',
                'Czechia':'Czech Republic',
                'Hong Kong SAR':'Hong Kong',
                'Macao SAR':'Macau'}

In [ ]:
df_group['Country/Region'] = df_group['Country/Region'].replace(country_dict)

In [ ]:
df_group['observation_date'] = pd.to_datetime(df_group.observation_date)
df_group.rename(columns={'Country/Region':'country'}, inplace=True)
df_group = df_group.sort_values(by=['country','observation_date'])
df_group['NormalizedStart'] = df_group.groupby(['country']).cumcount() + 1

In [ ]:
df_group['NormalizedFirstDeath'] = df_group[df_group.Deaths > 0].groupby('country').cumcount() + 1
df_group['NormalizedFirstDeath'] = df_group['NormalizedFirstDeath'].fillna(0)

In [ ]:
df_group.iloc[:,-5:] = df_group.iloc[:,-5:].astype(int)

In [ ]:
df_group['death_delta'] = df_group.Deaths.diff().fillna(0).astype(int)
df_group['confirmed_delta'] = df_group.Confirmed.diff().fillna(0).astype(int)
df_group.loc[df_group.NormalizedStart == 1, 'death_delta'] = 0
df_group.loc[df_group.NormalizedStart == 1, 'confirmed_delta'] = 0

In [ ]:
df_group = df_group.drop_duplicates(subset=['country','observation_date'])

In [ ]:
df_daily_cases = df_group[df_group.country != 'China'].groupby('observation_date', as_index=False)['confirmed_delta'].sum()

In [ ]:
df_daily_cases['rate_of_change'] = df_daily_cases.confirmed_delta.pct_change() + 1
df_daily_cases['cases_3DMA'] = df_daily_cases.confirmed_delta.rolling(3).mean()
df_daily_cases['rate_of_change_3DMA'] = df_daily_cases.cases_3DMA.pct_change() + 1
df_daily_cases

## Connect to and update Google sheets

In [ ]:
pycred = pygsheets.authorize(service_file='/Users/paulbrown/Documents/credentials.json')
#opening the gsheet and sheet you want to work with
ss = pycred.open('Corona Virus Dataset')[0]
#overwrite what is in the sheet with your df
ss.set_dataframe(df_group,(1,1))

In [ ]:
pycred = pygsheets.authorize(service_file='/Users/paulbrown/Documents/credentials.json')
#opening the gsheet and sheet you want to work with
ss = pycred.open('Corona Virus Dataset')[1]
#overwrite what is in the sheet with your df
ss.set_dataframe(df_daily_cases,(1,1))